In [ ]:
import os
import glob

sacred_dir = "sacred_variance"
parent_dir = os.getcwd()
print(os.getcwd())
test_json_files = glob.glob(os.path.join(parent_dir, sacred_dir, "test**/*info.json"), recursive=True)
print(len(test_json_files))

In [ ]:
import json

def exp_name_from_file(file):
    # test_10000_qvmix_uc_50_owf_25_9__2023-03-15_23-09-01__2023-03-21_06-07-53
    # test_10000_qvmix_uc_50_owf_25_cc_True_5__2023-04-01_23-05-55__2023-04-10_17-30-24
    exp_name = os.path.basename(os.path.dirname(file))
    exp_name = exp_name.split("_")[2:]

    if 'owf' in exp_name and 'dqn' in exp_name:
        index_env = 5
    elif "owf" in exp_name:
        index_env = 4
    else:
        index_env = 5

    if exp_name[index_env] != exp_name[2] and index_env is not 4:
        if int(exp_name[2]) == 10 and int(exp_name[index_env]) < 10:
            pass
        else:
            if not "dqn" in exp_name:
                print("merde")
                print(int(exp_name[2]) == 10, int(exp_name[index_env]) < 10)
                print(exp_name, exp_name[2], exp_name[index_env])
    if "cc_True" in file:
        exp_name = [exp_name[0], exp_name[index_env-1], exp_name[index_env], "cc_True"]
    else:
        exp_name = [exp_name[0], exp_name[index_env-1], exp_name[index_env], "cc_False"]
    if exp_name[1] == "sarl":
        exp_name[1]= "owf"
    return exp_name

def dict_results_from_json(test_json_files, method, env1, env2, cc):
    dict_all=[]

    for i in test_json_files:
        exp_name=exp_name_from_file(i)

        if cc and "cc_True" in i:
            pass
        elif not cc and ("cc_False" in i or "cc" not in i):
            pass
        else:
            continue

        if exp_name[0] == method and exp_name[1] == env1 and exp_name[2] == env2:
            exp_name = "_".join(exp_name)

            # Remove dqn too much
            if exp_name.startswith("dqn"):
                # ignore uc/ic 10 50 100 and owf 5 25 50
                if "10" in k or "50" in k or "100" in k or ("owf" in k and ('5' in k or '25' in k or '50' in k)):
                    continue
        else:
            continue
        try:
            with open(i, "r") as file:
                infos = json.load(file)
                dict_exp= {}
                dict_exp["name"] = os.path.basename(os.path.dirname(i))
                dict_exp["test_return_mean"] = [i["value"] for i in infos["test_return_mean"]]
                dict_exp["test_return_mean_T"] = infos["test_return_mean_T"]
                dict_exp["test_return_std"] = [i["value"] for i in infos["test_return_std"]]
                dict_all.append(dict_exp)
        except Exception as e:
            print("Exception",i, e)
    return dict_all

methods = ["qmix", "qvmix",  "coma", "iql", "qplex", "dqn", "facmac"]
env1 = ["uc", "c", "owf"]
env2 = {"uc":["3", "5", "10", "50", "100" ], "c":["3", "5", "10", "50", "100"], "owf":["1","2","5","25","50"]}
cc = [True, False]
results={}
for cc_ in cc:
    for i in methods:
        for j in env1:
            for k in env2[j]:
                exp_name = f"{i}_{j}_{k}_cc_{cc_}"
                results[exp_name] = dict_results_from_json(test_json_files, i,j,k, cc_)
print(len(results["facmac_owf_50_cc_False"]))

In [ ]:
for i in results["facmac_owf_50_cc_False"]:
    if i["test_return_mean_T"][-1]<2000000:
        print("rm -r ", i["name"])

# DO NOT FORGET TO SET cc_str = "cc_False" or "cc_True"

In [ ]:
list_100=[]
list_1000=[]
list_10000=[]
for i in results["facmac_owf_50_cc_False"]:
    if i['name'].startswith("test_100_"):
        list_100.append(i)
    elif i['name'].startswith("test_1000_"):
        list_1000.append(i)
    elif i['name'].startswith("test_10000_"):
        list_10000.append(i)
print(len(list_1O0), len(list_1000), len(list_10000))

In [ ]:
import numpy as np
# Check max diff between means
list_mean = [i["test_return_mean"] for i in list_100]
list_mean = np.array(list_mean)
list_diff = np.max(list_mean, axis=0) - np.min(list_mean, axis=0)
print(np.max(list_diff))
# print the value of the min and max also where this max list diff is met
print(np.min(list_mean, axis=0)[np.argmax(list_diff)])

list_mean = [i["test_return_mean"] for i in list_1000]
list_mean = np.array(list_mean)
list_diff = np.max(list_mean, axis=0) - np.min(list_mean, axis=0)
print(np.max(list_diff))
print(np.min(list_mean, axis=0)[np.argmax(list_diff)])

list_mean = [i["test_return_mean"] for i in list_10000]
list_mean = np.array(list_mean)
list_diff = np.max(list_mean, axis=0) - np.min(list_mean, axis=0)
print(np.max(list_diff))
print(np.min(list_mean, axis=0)[np.argmax(list_diff)])

In [ ]:
from matplotlib.ticker import ScalarFormatter
from scipy.ndimage import gaussian_filter1d
import numpy as np
from matplotlib import pyplot as plt


plt.style.use('seaborn-white')
fig, axes = plt.subplots(1,3)
fig.set_figheight(20)
fig.set_figwidth(120)
plt.subplots_adjust(wspace=0.17)


for i in list_100:
    axes[0].plot(i["test_return_mean_T"], gaussian_filter1d(i["test_return_std"], sigma=10), lw=5, label=i["name"])
    #axes[0].plot(i["test_return_mean_T"], gaussian_filter1d(i["test_return_mean"], sigma=10), lw=5, label=i["name"])

for i in list_1000:
    axes[1].plot(i["test_return_mean_T"], gaussian_filter1d(i["test_return_std"], sigma=10), lw=5, label=i["name"])
    #axes[1].plot(i["test_return_mean_T"], gaussian_filter1d(i["test_return_mean"], sigma=10), lw=5, label=i["name"])

for i in list_10000:
    axes[2].plot(i["test_return_mean_T"], gaussian_filter1d(i["test_return_std"], sigma=10), lw=5, label=i["name"])
    #axes[2].plot(i["test_return_mean_T"], gaussian_filter1d(i["test_return_mean"], sigma=10), lw=5, label=i["name"])
for ax in axes:
    ax.grid()
    ax.set_xlim(0, 2000000)
    fig.canvas.draw()
    if ax == axes[0]:
        ax.set_ylabel('Standard deviation of R_0', fontsize=80)
    ax.yaxis.set_tick_params(length=15, width=5)
    ax.xaxis.set_tick_params(length=15, width=5)
    fig.canvas.draw()
    a = ax.get_xticks()
    ax.set_xticks(np.arange(min(a), max(a)+1, 0.5*1000000))
    ax.set_xticks(ax.get_xticks()) # to remove a warning
    ax.set_xticklabels(['0', '0.5', '1', '1.5', '2'], fontsize=80)
# titles
axes[0].set_title('100 episodes', fontsize=80)
axes[1].set_title('1,000 episodes', fontsize=80)
axes[2].set_title('10,000 episodes', fontsize=80)

plt.rc('font', size=80)
axes[1].set_xlabel('Timesteps ('+r'$\times \mathregular{10^{6}}$' + ')', fontsize=80)

plt.plot()
plt.savefig(f"results_scripts/plots/variance_analysis.pdf", bbox_inches='tight')
print("")
